### ColBERT

Получение косинусной близости для ранжирования и матчинга без использования преимуществ ColBERT'а, так же как ранее использовался обычный BERT. Как используется ColBERT в оригинальном виде через FAISS index и быстрый поиск, можно посмотреть в tutorial_by_index.ipynb

### Загрузка нейронной сети 

In [1]:
from interface import top_n_similar, load_model, get_query_emb_batch
import numpy as np

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/2801_lr04_bsize_210_samsung/none/2024-06/17/07.43.04/checkpoints/colbert-51-finish"

# device = "cpu"
device = "cuda" 

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию.  

checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

### Получаем и сохраняем эмбеддинги офферов и моделей на диск

- ColBERT bert-base-multilingual-cased_dim_768: размер одного эмбеддинга на диске 98.4 Кб, после усреднения по токенам 3.2 Кб

- ColBERTv2.0_dim_128: размер одного эмбеддинга на диске 16.5 Кб, после усреднения по токенам 640 байт


In [2]:
dst_fld = "/home/sondors/Documents/price/ColBERT/tutorial_emb"

offers = ['Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
        'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
        'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)']

offer_embs = get_query_emb_batch(offers, checkpoint, batch_size=100, batch_size2=1000)
for i in range(len(offers)):
    np.save(f'{dst_fld}/{i}.npy', offer_embs[i])

batch: 3/3


In [3]:
category_id = 510401

model_ids = [5144478,
            410416,
            4509801,
            631587,
            5144477]

models = ['Samsung Galaxy Tab S8',
        'Starway Andromeda S8',
        'Samsung Galaxy Tab S7 11 128Gb',
        'Haier G781-S',
        'Samsung Galaxy Tab S8+']

model_embs = get_query_emb_batch(models, checkpoint, batch_size=100, batch_size2=1000)

for i in range(len(models)):
    np.save(f'{dst_fld}/{category_id}_{model_ids[i]}.npy', model_embs[i])

batch: 5/5


In [4]:
print(model_embs[0])
print(np.shape(model_embs[0]))
print(type(model_embs[0]))

[ 0.00506575  0.03047421 -0.00769766 -0.12129673 -0.03638392 -0.01104426
 -0.07106609  0.09459791  0.04582676  0.02620405  0.12076786  0.12187437
 -0.02819399  0.1758198   0.03564505  0.04535684 -0.00567103 -0.14024694
 -0.16135938 -0.00535819 -0.0038752   0.01934042  0.09791345 -0.14434579
  0.07704479  0.00426503 -0.08078092  0.0210305  -0.01082102  0.05117488
 -0.1582363   0.02909349  0.07359951 -0.00300251  0.00554421 -0.03901983
  0.01297752 -0.01381038 -0.03338131  0.06412245  0.04444331 -0.04440967
  0.02803332 -0.17791878 -0.10531406  0.00588788 -0.01402213 -0.11777657
  0.06998768 -0.02943917  0.05043336 -0.01284498  0.01364657 -0.03303193
 -0.00049819  0.06858405 -0.03416084 -0.0097131   0.03166019 -0.05174951
 -0.13222457  0.00570185  0.15629141  0.00633475 -0.07199998  0.09725697
 -0.06448828  0.08114342  0.0341798   0.02909898 -0.12092433  0.12160192
  0.08329911  0.00749979  0.04096789 -0.08939541 -0.05515286  0.05079212
 -0.01087672  0.08976662  0.08374695 -0.00879887 -0

### Загружаем эмбеддинги с диска в память

In [5]:
offer_embs = []
for i in range(len(offers)):
    offer_embs.append(np.load(f'{dst_fld}/{i}.npy'))
model_embs = []
for i in range(len(models)):
    model_embs.append(np.load(f'{dst_fld}/{category_id}_{model_ids[i]}.npy'))

In [6]:
print(offer_embs[0])
print(np.shape(offer_embs[0]))
print(type(offer_embs[0]))

[ 0.00206919 -0.00732165 -0.06499577 -0.16439947 -0.03485534 -0.01748732
 -0.0653784   0.0228113   0.04654867  0.02284213  0.05694153  0.12334426
 -0.01263027  0.1089011   0.03864664  0.04623519  0.05939067 -0.12963913
 -0.13958208  0.00669574 -0.03337958 -0.03243835  0.05666415 -0.08137263
  0.02321253  0.00628771 -0.04938281 -0.01213988 -0.01977007  0.06275578
 -0.13023445  0.0094604   0.00563033 -0.03194131  0.03756712 -0.07029375
  0.00472204  0.0047655  -0.02077439  0.00347974  0.04877551 -0.02081151
  0.02516995 -0.10300693 -0.10019424 -0.018432   -0.03014309 -0.09664342
  0.05062846 -0.03886686 -0.00172609 -0.03690704  0.02370398 -0.01243838
  0.02377371  0.07620025 -0.01966556  0.0101718  -0.0105662  -0.02978829
 -0.09270223  0.04177877  0.13209578 -0.00695268 -0.0892677   0.02582307
 -0.04905206  0.09788479  0.05553973 -0.03519689 -0.06844375  0.04882978
  0.05719822  0.03906595  0.02336171 -0.11063568 -0.0220468   0.03617341
  0.00838273  0.08481611  0.11760835 -0.04036083 -0

In [7]:
print(model_embs[0])
print(np.shape(model_embs[0]))
print(type(model_embs[0]))

[ 0.00506575  0.03047421 -0.00769766 -0.12129673 -0.03638392 -0.01104426
 -0.07106609  0.09459791  0.04582676  0.02620405  0.12076786  0.12187437
 -0.02819399  0.1758198   0.03564505  0.04535684 -0.00567103 -0.14024694
 -0.16135938 -0.00535819 -0.0038752   0.01934042  0.09791345 -0.14434579
  0.07704479  0.00426503 -0.08078092  0.0210305  -0.01082102  0.05117488
 -0.1582363   0.02909349  0.07359951 -0.00300251  0.00554421 -0.03901983
  0.01297752 -0.01381038 -0.03338131  0.06412245  0.04444331 -0.04440967
  0.02803332 -0.17791878 -0.10531406  0.00588788 -0.01402213 -0.11777657
  0.06998768 -0.02943917  0.05043336 -0.01284498  0.01364657 -0.03303193
 -0.00049819  0.06858405 -0.03416084 -0.0097131   0.03166019 -0.05174951
 -0.13222457  0.00570185  0.15629141  0.00633475 -0.07199998  0.09725697
 -0.06448828  0.08114342  0.0341798   0.02909898 -0.12092433  0.12160192
  0.08329911  0.00749979  0.04096789 -0.08939541 -0.05515286  0.05079212
 -0.01087672  0.08976662  0.08374695 -0.00879887 -0

### Получаем выдачу топ N моделей для каждого оффера

In [8]:
top_n = top_n_similar(offer_embs, model_embs, model_ids, batch_size = 1000, n=5)

for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['cosine_sims'][j]
        print(f"\t{id}: {models[model_ids.index(id)]} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 0.9
	5144477: Samsung Galaxy Tab S8+ --> 0.66
	4509801: Samsung Galaxy Tab S7 11 128Gb --> 0.41
	410416: Starway Andromeda S8 --> 0.38
	631587: Haier G781-S --> 0.1
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 0.9
	5144477: Samsung Galaxy Tab S8+ --> 0.6
	4509801: Samsung Galaxy Tab S7 11 128Gb --> 0.48
	410416: Starway Andromeda S8 --> 0.41
	631587: Haier G781-S --> 0.11
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 0.9
	5144478: Samsung Galaxy Tab S8 --> 0.61
	410416: Starway Andromeda S8 --> 0.24
	4509801: Samsung Galaxy Tab S7 11 128Gb --> 0.22
	631587: Haier G781-S --> 0.07
_______________________________________________________

In [9]:
print(top_n)

[{'model_ids': [5144478, 5144477, 4509801, 410416, 631587], 'cosine_sims': array([0.89728355, 0.6588423 , 0.40546757, 0.38076147, 0.09940813],
      dtype=float32)}, {'model_ids': [5144478, 5144477, 4509801, 410416, 631587], 'cosine_sims': array([0.902261  , 0.6040057 , 0.47631758, 0.41486305, 0.11185384],
      dtype=float32)}, {'model_ids': [5144477, 5144478, 410416, 4509801, 631587], 'cosine_sims': array([0.8963696 , 0.6139134 , 0.24416839, 0.22016335, 0.07392624],
      dtype=float32)}]
